# 0 - Import packages:

In [2]:
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc
from sklearn.model_selection import StratifiedKFold
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 5.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 - Load data:

## 1.1 - Train set: 

In [3]:
df_data_1 = pd.read_csv("../common/albertom/train_test_split/X_train_pca.csv", index_col="Unnamed: 0")
# df_data_1 = df_data_1.head(500)
print(df_data_1.shape)
df_data_1.head()

(22851, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,29.479667,5.668109,-0.209441,-1.345416,-2.547592,-0.433499,-0.295371,-1.271754,-1.085988,-1.853838,-0.164936,0.076310,-0.171494,-0.263780,0.252866
1,-8.119109,-0.918377,3.104810,-0.575573,-2.197244,-0.918233,-3.359054,1.598714,-1.490960,0.341960,1.479482,0.113068,-0.044529,-1.130631,0.348799
2,-12.897545,8.760454,2.352287,-0.329169,0.227234,-0.455962,-3.328908,0.392441,2.903130,-1.738854,0.510227,-1.106186,0.176982,0.568320,-1.206535
3,3.210958,-1.071009,1.016989,-0.326063,-0.583567,-0.108464,-0.159468,-0.904758,1.466309,0.394476,0.082506,-0.034436,-0.178566,-1.071025,-0.518600
4,21.015684,-0.687015,-0.105652,-1.678955,2.238427,-2.418710,-3.161304,-0.056711,1.178904,2.322390,2.085181,2.039296,-1.051172,0.075783,-0.014417


## 1.2 - Train labels:

In [4]:
df_target_data_1 = pd.read_csv("../common/albertom/train_test_split/y_train.csv", index_col="Unnamed: 0")
# df_target_data_1 = df_target_data_1.head(500)
print(df_target_data_1.shape)
df_target_data_1.head()

(22851, 2)


,0,1
0,0,34188358
1,0,41229741
2,0,43708758
3,0,43058589
4,1,25456031


## 1.3 - Test set:

In [5]:
df_data_2 = pd.read_csv("../common/albertom/train_test_split/X_test_pca.csv", index_col="Unnamed: 0")
# df_data_2 = df_data_2.head(100)
print(df_data_2.shape)
df_data_2.head()

(4033, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-1.718190,-6.316477,-2.083559,0.975329,-0.296123,1.974713,0.379124,-0.007346,1.958610,-1.165546,0.759263,-0.382959,-0.584112,1.553049,0.568266
1,0.020365,-0.602705,9.139554,0.840974,0.546386,-5.230693,-0.084946,-10.231455,-0.852961,0.451181,0.669960,-0.829125,1.024185,0.229763,0.039209
2,9.975737,-3.348945,-0.572407,0.087099,0.788437,0.297061,0.889188,0.581228,-0.122152,-0.471265,0.488114,-0.081485,-0.171996,-0.577770,-0.305709
3,-22.105474,0.258111,0.720882,-3.436382,-1.524025,0.087214,-0.444141,0.608868,1.049485,0.409754,-0.329480,-0.048176,-0.009099,-0.527233,1.436252
4,-14.011642,2.771666,-3.670775,-0.453639,-0.303950,-1.095188,-2.580395,-1.656323,-1.115220,0.806742,-0.176123,0.275889,-0.784060,1.083732,-1.022600


## 1.4 - Test labels:

In [6]:
df_target_data_2 = pd.read_csv("../common/albertom/train_test_split/y_test.csv", index_col="Unnamed: 0")
# df_target_data_2 = df_target_data_2.head(100)
print(df_target_data_2.shape)
df_target_data_2.head()

(4033, 2)


,0,1
0,1,10930294
1,1,55016784
2,0,48129536
3,1,37805023
4,0,22798417


# 2 - Globals:

## 2.1 - Parameters 

In [55]:
n_folds = 3
num_steps = 3001

valid_size = df_data_1.shape[0]/n_folds
batch_size = 128

num_features = train_dataset.shape[1]
num_examples = train_dataset.shape[0]
distinct_labels = len(df_target_data_1["0"].unique())

train_dataset = df_data_1.values
test_dataset = df_data_2.values

split_labels = df_target_data_1["0"].values
train_labels = (np.arange(distinct_labels) == split_labels[:,None]).astype(np.float32)

test_labels = df_target_data_2["0"].values
test_labels = (np.arange(distinct_labels) == test_labels[:,None]).astype(np.float32)

## 2.2 - Tensorflow graph: 

In [51]:
graph = tf.Graph()
with graph.as_default():

    # Input data.
    X = tf.placeholder(tf.float32, shape=(batch_size, num_features))
    t = tf.placeholder(tf.float32, shape=(batch_size, distinct_labels))
    X_unseen = tf.placeholder(tf.float32, shape=(valid_size, num_features))
    
    L2_reg = tf.placeholder(tf.float32, shape=[])
    learning_rate = tf.placeholder(tf.float32, shape=[])

    # Variables.
    num_hidden1 = 10
    num_hidden2 = 7
    
    W1 = tf.Variable(tf.truncated_normal([num_features, num_hidden1]) )
    b1 = tf.Variable(tf.zeros([num_hidden1]))

    W2 = tf.Variable(tf.truncated_normal([num_hidden1, distinct_labels]))
    b2 = tf.Variable(tf.zeros([distinct_labels]))
    
    # Training.
    h1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    logits = tf.matmul(h1, W2) + b2
    
    # Loss NO reg.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, t))
    regularization = (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2))
    loss = loss + L2_reg * regularization

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

    # Predictions for training, validation.
    train_predictions = tf.nn.softmax(logits)
    unseen_predictions = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(X_unseen, W1) + b1), W2) + b2)

## 2.3 - Functions:

In [62]:
# Given a set of model parameters (learning rate, regularization penalty coefficient), find the optimal parameters with cross validation

def cross_validation(set_of_learning_rates, set_of_regs, X_train, y_train):
    
    # Get train-validation set stratified (keeps the same distribution) splitter
    skf = StratifiedKFold(n_splits=3)
    set_of_params = [(x,y) for x in set_of_learning_rates for y in set_of_regs]
    
    print("Number of folds: " + str(n_folds))
    print("Number of parameters combinations: " + str(len(set_of_params)))
    print("We will train " + str(n_folds * len(set_of_params)) + " neural networks for this cv task.")
    
    f1 = np.zeros(len(set_of_params))

    # Repeat for every train-validate combination 
    for cv_train_index, cv_val_index in skf.split(X_train, split_labels):
            
        X_cv_train, X_cv_val = X_train[cv_train_index], X_train[cv_val_index]
        y_cv_train, y_cv_val = y_train[cv_train_index], y_train[cv_val_index]
        with tf.Session(graph=graph) as session:
            for param_idx, param in enumerate(set_of_params):
                
                print(str(param_idx) + " ) Combination: [Parameters: " +  str(param) + " | Train model]")
                for step in np.arange(num_steps):
                    
                    offset = (step * batch_size) % (num_examples - batch_size)
                
                    X_batch = X_cv_train[offset:(offset + batch_size), :]
                    t_batch = y_cv_train[offset:(offset + batch_size)]
                    feed_dict = {
                        X : X_batch,
                        t : t_batch,
                        X_unseen : X_cv_val,
                        L2_reg : param[1],
                        learning_rate : param[0]
                    }
                    _, l, pred_batch = session.run( [optimizer, loss, train_predictions], feed_dict=feed_dict)
                    
                    if (step % 500 == 0):
                        print("> Minibatch loss at step %d: %f" % (step, l))
                        print("> Minibatch f1: %.1f%%" % f1(pred_batch, t_batch))
                        print("> Validation f1: %.1f%%" % f1(unseen_predictions.eval(), y_cv_val) )
                
                f1_cv = f1(valid_predictions.eval(), y_cv_val)
                print("|> Final Validation f1: %.1f%%" % f1_cv)
                f1[param_idx] = f1[param_idx] + f1_cv

    f1 = f1 / n_folds # Calculate average error
    

    print(">>>>>> Best [learning rate - f1] = ", set_of_params[np.argmax(f1)][1])
    print(">>>>>> BEST [reg - f1] = ", set_of_params[np.argmax(f1)][0])

    return set_of_params[np.argmax(f1)]

In [53]:
def f1(predicted_classes, real_classes):
    return f1_score(real_classes, predicted_classes)

# 3 - Training: 

## 3.1 - Cross validation for optimal hyperparameters:

In [63]:
set_of_learning_rates = np.arange(0.01, 0.03, step=0.01)
set_of_L2_regs = np.arange(0.01, 0.03, step=0.01)

print("Cross-validation on TRAINING-SET for [learning rate, L2_reg] ...")

best_params = cross_validation(set_of_learning_rates, set_of_L2_regs, train_dataset, train_labels)

best_learning_rate = best_params[0]
best_L2_reg = best_params[1]

print("Best parameters: [" + str(best_learning_rate) + ", " + str(best_L2_reg) + "] ...")

Cross-validation on TRAINING-SET for [learning rate, L2_reg] ...
Number of folds: 3
Number of parameters combinations: 4
We will train 12 neural networks for this cv task.
0 ) Combination: [Parameters: (0.01, 0.01) | Train model]
{<tf.Tensor 'Placeholder:0' shape=(128, 15) dtype=float32>: array([[ -6.83051539e+00,  -2.22850216e+00,   1.37965579e+00, ...,
          3.85150092e-01,   1.91536103e-01,   3.11393157e-02],
       [ -6.30360006e+00,  -8.34043056e+00,   3.27378055e+00, ...,
          4.23582951e-01,   1.62311207e+00,   7.01300056e-01],
       [ -2.89803263e+00,  -3.78699029e+00,   2.61338417e+00, ...,
         -5.86965073e-02,  -6.59344101e-01,  -2.01150201e-01],
       ..., 
       [ -1.92743804e+01,   3.10216295e+00,  -5.20942610e+00, ...,
          4.88862244e-01,  -2.73151115e-01,   1.30383369e+00],
       [ -3.50817104e+00,   9.91754419e+00,  -1.00857643e+01, ...,
         -1.15110190e+00,  -2.61148882e-01,  -1.61840275e-01],
       [  1.30875252e+01,  -2.57062681e+00,   1

FailedPreconditionError: Attempting to use uninitialized value Variable_3
	 [[Node: Variable_3/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_3"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_3)]]

Caused by op 'Variable_3/read', defined at:
  File "/Users/albertomariopirovano/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/albertomariopirovano/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/albertomariopirovano/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/albertomariopirovano/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/albertomariopirovano/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/albertomariopirovan

## 3.2 - Prediction using optimal hyperparameters:

In [ ]:
print("Prediction for TEST-SET using best parameters ...")

with tf.Session(graph=graph) as session:
    for param_idx, param in enumerate(set_of_params):
                
    print(str(param_idx) + " ) Combination: [Parameters: " +  str(param) + " | Train model]")
        for step in np.arange(num_steps):
                    
            X_batch = X_cv_train[offset:(offset + batch_size), :]
            t_batch = y_cv_train[offset:(offset + batch_size), :]
            offset = (step * batch_size) % (num_examples - batch_size)
            feed_dict = {
                X : X_batch,
                t : t_batch,
                X_unseen : test_dataset,
                L2_reg : best_L2_reg,
                learning_rate : best_learning_rate
            }
            if (step % 500 == 0):
                print("> Minibatch loss at step %d: %f" % (step, l))
                print("> Minibatch f1: %.1f%%" % f1(pred_batch, t_batch))
                print("> Validation f1: %.1f%%" % f1(unseen_predictions.eval(), y_cv_val) )
                
        f1_test = f1(unseen_predictions.eval(), test_labels)
        print("|> Test f1: %.1f%%" % f1_test)